## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [ ]:
# File location and type
file_location = "/FileStore/tables/data_f.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df = df.drop('summary')
df.show()

+--------------------+--------------------+
|             overall|          reviewText|
+--------------------+--------------------+
|                   5|The stained glass...|
|                   5|My 11 y.o. loved ...|
|Dragons and Wizar...| that make it ""s...|
|Even the perfecti...| so it tends to b...|
|                   5|The pictures are ...|
|                   5|I absolutely love...|
|                   5|          I love it!|
|                   5|MY HUSBAND LOVED ...|
|                   5|             love it|
|                   4|                cool|
|                   5|Exactly as descri...|
|                   5|Sometimes you nee...|
|I love all of the...| Great 30 minute toy|
|                   5|These little book...|
|                   2|This is indeed a ...|
|           Why? Well| there's really n...|
|Not recommended u...|                null|
|           Plus side| small enough to ...|
|                   4|I bought several ...|
|                   1|total wast

In [ ]:
from pyspark.sql.functions import col

df = df.filter(~col("overall").rlike('\D'))
df.show(5)

+-------+--------------------+
|overall|          reviewText|
+-------+--------------------+
|      5|The stained glass...|
|      5|My 11 y.o. loved ...|
|      5|The pictures are ...|
|      5|I absolutely love...|
|      5|          I love it!|
+-------+--------------------+
only showing top 5 rows



In [ ]:
# Load the data
from pyspark.sql.functions import col

data = df.select(col("reviewText"), col("overall"))

In [ ]:
from pyspark.sql.functions import col, sum

# Count the number of null values in each column
null_counts = data.select([sum(col(c).isNull().cast("int")).alias(c) for c in data.columns])

# Display the result
null_counts.show()

+----------+-------+
|reviewText|overall|
+----------+-------+
|         9|      0|
+----------+-------+



In [ ]:
# drop any rows with null values
data = data.na.drop()

In [ ]:
from pyspark.sql.functions import col, sum

# Count the number of null values in each column
null_counts = data.select([sum(col(c).isNull().cast("int")).alias(c) for c in data.columns])

# Display the result
null_counts.show()


+----------+-------+
|reviewText|overall|
+----------+-------+
|         0|      0|
+----------+-------+



In [ ]:
from pyspark.sql.functions import rand

# Assuming you have a DataFrame named 'df' with the dataset

# Specify the fraction of rows to delete
fraction_to_delete = 0.45  # 50% of the rows will be deleted

# Randomly delete rows
randomly_deleted_df = data.sample(fraction=1 - fraction_to_delete, seed=42)

# Display the remaining DataFrame
remaining_df = randomly_deleted_df
remaining_df.show()

# If you want to overwrite the original DataFrame with the remaining one, you can reassign the variable:
data = remaining_df

+--------------------+-------+
|          reviewText|overall|
+--------------------+-------+
|My 11 y.o. loved ...|      5|
|I absolutely love...|      5|
|MY HUSBAND LOVED ...|      5|
|                cool|      4|
|These little book...|      5|
|I bought several ...|      4|
|This is pretty mu...|      3|
|its a cute little...|      4|
|They were ok but ...|      2|
|Great fun for my ...|      5|
|          great book|      5|
|   Grandson loved it|      5|
|This is a small book|      5|
|"i think there ar...|      3|
|"This was a great...|      5|
|I bought this as ...|      4|
|Not sure why ther...|      2|
|We bought this to...|      4|
|This book is smal...|      5|
|My 4 year old lov...|      5|
+--------------------+-------+
only showing top 20 rows



In [ ]:
data.count()

Out[27]: 1006439

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml import Pipeline

# Define the tokenizer
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")

# Define the stop words remover
stop_words_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

# Define the hashing term frequency (HTF) vectorizer
hashing_tf = HashingTF(inputCol="filtered_words", outputCol="raw_features")

# Define the inverse document frequency (IDF) transformer
idf = IDF(inputCol="raw_features", outputCol="features")

# Define the preprocessing pipeline
preprocessing_pipeline = Pipeline(stages=[tokenizer, stop_words_remover, hashing_tf, idf])

# Fit the preprocessing pipeline to the data and transform the data
preprocessed_data = preprocessing_pipeline.fit(data).transform(data).select("features", "overall")


In [ ]:
from pyspark.sql.functions import col

# Convert the 'overall' column to a numeric type
preprocessed_data = preprocessed_data.withColumn('overall', col('overall').cast('double'))

# Check the schema again
preprocessed_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- overall: double (nullable = true)



In [ ]:
preprocessed_data = preprocessed_data.repartition(5000)
(training_data, test_data) = preprocessed_data.randomSplit([0.7, 0.3], seed=42)

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Define the Naive Bayes model
nb = NaiveBayes(labelCol="overall", smoothing=1.0)

# Cache the training data to avoid recalculating it multiple times
training_data.cache()

# Fit the model to the training data
nb_model = nb.fit(training_data)

# Unpersist the training data to release memory
training_data.unpersist()

# Make predictions on the test data
predictions = nb_model.transform(test_data)

# Evaluate the model using accuracy metric
evaluator = MulticlassClassificationEvaluator(labelCol="overall", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

Accuracy: 0.09492557487733852


In [ ]:
from pyspark.sql.functions import col

# Convert the 'overall' column to a numeric type
data = data.withColumn('overall', col('overall').cast('double'))

# Check the schema again
data.printSchema()

root
 |-- reviewText: string (nullable = true)
 |-- overall: double (nullable = true)



In [ ]:
from pyspark.ml.feature import Word2Vec, RegexTokenizer
from pyspark.sql.functions import col
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Define a regular expression tokenizer to split the text into words
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="word", pattern="\\W")

# Apply the tokenizer to the data
words_df = tokenizer.transform(data).na.drop(subset=["word"])

# Check if there are any null or missing values in the "word" column
if words_df.filter(col("word").isNull() | (col("word") == "")).count() > 0:
    print("Warning: Null or missing values found in the 'word' column. Please handle them before proceeding.")

# Learn a Word2Vec model on the text data
word2vec = Word2Vec(vectorSize=100, minCount=5, inputCol="word", outputCol="features")
word2vec_model = word2vec.fit(words_df)

# Transform the data using the Word2Vec model
word2vec_df = word2vec_model.transform(words_df)

# Split the data into training and test sets
(training_data, test_data) = word2vec_df.randomSplit([0.7, 0.3])

# Train a Logistic Regression model on the data
lr = LogisticRegression(labelCol="overall", maxIter=10, regParam=0.01, elasticNetParam=0)
lr_model = lr.fit(training_data)

# Make predictions on the test data
predictions = lr_model.transform(test_data)

# Evaluate the model using accuracy metric
evaluator = MulticlassClassificationEvaluator(labelCol="overall", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-931624360125395>:14
     10 # Apply the tokenizer to the data
     11 words_df = tokenizer.transform(data).na.drop(subset=["word"])
     13 # Check if there are any null or missing values in the "words" column
---> 14 # Check if there are any null or missing values in the "word" column
     15 if words_df.filter(col("word").isNull() | col("word").isEmpty()).count() > 0:
     16     print("Warning: Null or missing values found in the 'word' column. Please handle them before proceeding.")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     5

In [ ]:
# Compute confusion matrix
predictionAndLabels = predictions.select("prediction", "overall").rdd.map(lambda x: (x.prediction, x.overall))
metrics = MulticlassMetrics(predictionAndLabels)
confusion_matrix = metrics.confusionMatrix().toArray()
print("Confusion Matrix:")
print(confusion_matrix)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-931624360125386>:2
      1 # Compute confusion matrix
----> 2 predictionAndLabels = predictions.select("prediction", "overall").rdd.map(lambda x: (x.prediction, x.overall))
      3 metrics = MulticlassMetrics(predictionAndLabels)
      4 confusion_matrix = metrics.confusionMatrix().toArray()

NameError: name 'predictions' is not defined